# Analizador Inteligente de Currículums con Snowflake y OCR

## ¿Qué es OCR y por qué es útil para procesar currículums?

El Reconocimiento Óptico de Caracteres (OCR) es una tecnología que permite convertir diferentes tipos de documentos, como imágenes escaneadas, archivos PDF o fotografías de documentos, en datos editables y con capacidad de búsqueda. En el contexto de los currículums vitae, el OCR es particularmente valioso porque:

- **Automatiza la extracción de información**: Permite procesar grandes volúmenes de CVs sin intervención manual.
- **Facilita la búsqueda**: Convierte documentos que antes eran solo imágenes en texto sobre el que podemos realizar consultas.
- **Habilita el análisis avanzado**: Posibilita la aplicación de técnicas de NLP y machine learning sobre el contenido extraído.

## Arquitectura RAG: Mejorando las respuestas con contexto

Este proyecto utiliza la arquitectura RAG (Retrieval-Augmented Generation) que combina:

1. **Retrieval (Recuperación)**: Usando vectores de embedding para encontrar los fragmentos más relevantes de los currículums.
2. **Augmentation (Aumento)**: Proporcionando este contexto relevante al modelo de lenguaje.
3. **Generation (Generación)**: Utilizando un modelo de lenguaje para generar respuestas precisas basadas en el contexto.


In [ ]:
# Importar paquetes de Python
import streamlit as st
import tesserocr
import io
from PIL import Image
import pandas as pd  # Para análisis de datos extraídos

# Utilizamos Snowpark
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.types import StringType, StructField, StructType, IntegerType
from snowflake.snowpark.files import SnowflakeFile
from snowflake.core import CreateMode
from snowflake.core.table import Table, TableColumn
from snowflake.core.schema import Schema
from snowflake.core import Root

session = get_active_session()
session.use_schema("curriculum_schema")
root = Root(session)
database = root.databases[session.get_current_database()]

In [ ]:
#Creamos la tabla que va a alojar los distintos chunks y los vectores para hacer el retrieval
cvs_chunks_table = Table(
    name="cvs_chunks_table",
    columns=[
        TableColumn(name="relative_path", datatype="string"),
        TableColumn(name="file_url", datatype="string"),
        TableColumn(name="scoped_file_url", datatype="string"),
        TableColumn(name="chunk", datatype="string"),
        TableColumn(name="chunk_vec", datatype="vector(float,768)")
    ]
)
database.schemas["curriculum_schema"].tables.create(cvs_chunks_table, mode=CreateMode.or_replace)

In [ ]:
#Cremos la clase que procesa el ocr con tesserocr y la registres como una user defined table function que podamos aplicar de forma iterativa para cada url/stage file
class CvText:
    def process(self, file_url: str):
        with SnowflakeFile.open(file_url, 'rb') as f:
            buffer = io.BytesIO(f.readall())
        image = Image.open(buffer)
        text = tesserocr.image_to_text(image)
        yield (text,)  

output_schema = StructType([
    StructField("full_text", StringType())
])

session.udtf.register(
    CvText,
    name="CV_TEXT",
    is_permanent=True,
    stage_location="@curriculum_schema.cvs_to_ocr",
    schema="curriculum_schema",
    output_schema=output_schema,
    packages=["tesserocr", "pillow", "snowflake-snowpark-python"],
    replace=True
)

In [ ]:
--Mostramos el uso de la función creada sobre cada cv del stage
SELECT 
    relative_path, 
    file_url, 
    build_scoped_file_url(@curriculum_schema.cvs_to_ocr, relative_path) AS scoped_file_url,
    ocr_result.full_text,
FROM 
    directory(@curriculum_schema.cvs_to_ocr),
    TABLE(CV_TEXT(build_scoped_file_url(@curriculum_schema.cvs_to_ocr, relative_path))) AS ocr_result;


In [ ]:
--Creamos una tabla intermedia que aloje los resultados obtenidos del ocr y que vectorizaremos e insertaremos en la tabla fija en el siguiente bloque
CREATE OR REPLACE TEMPORARY TABLE temp_ocr_results AS
SELECT 
    relative_path, 
    file_url, 
    build_scoped_file_url(@curriculum_schema.cvs_to_ocr, relative_path) AS scoped_file_url,
    ocr_result.full_text,
FROM 
    directory(@curriculum_schema.cvs_to_ocr),
    TABLE(CV_TEXT(build_scoped_file_url(@curriculum_schema.cvs_to_ocr, relative_path))) AS ocr_result;

In [ ]:
-- Insertamos los chunks con vectorización en la tabla fija
DELETE FROM cvs_chunks_table;
INSERT INTO cvs_chunks_table (
    relative_path, 
    file_url, 
    scoped_file_url, 
    chunk, 
    chunk_vec
)
SELECT 
    relative_path, 
    file_url,
    scoped_file_url,
    chunk.value,
    SNOWFLAKE.CORTEX.EMBED_TEXT_768('snowflake-arctic-embed-m', chunk.value) AS chunk_vec -- usamos el modelo de embedding disponible de snowflake para vectorizar en vectores de 768 dimensiones el contenido de cada chunk

FROM
    temp_ocr_results,
    LATERAL FLATTEN(SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(full_text,'none', 2000, 300)) chunk;

In [ ]:
import streamlit as st
import pandas as pd

# Parámetros configurables
model = "mistral-7b"
num_chunks = 10  #fragmentos de contexto, no igual a cvs (10 porque se han generado 10 chunks de los cvs del stage)

def create_prompt(pregunta):
    # Consulta para recuperar contexto
    cmd = """
     WITH results AS
     (SELECT RELATIVE_PATH,
       VECTOR_COSINE_SIMILARITY(cvs_chunks_table.chunk_vec,
                SNOWFLAKE.CORTEX.EMBED_TEXT_768('snowflake-arctic-embed-m', ?)) as similarity,
       chunk
     FROM cvs_chunks_table
     ORDER BY similarity DESC
     LIMIT ?)
     SELECT chunk, relative_path, similarity FROM results 
     """
    df_context = session.sql(cmd, params=[pregunta, num_chunks]).to_pandas()      
    
    # Recolectamos todos los paths únicos de los archivos usados
    unique_paths = df_context['RELATIVE_PATH'].unique()
    
    # Creamos contexto para el prompt
    prompt_context = " ".join(df_context['CHUNK'].tolist())
    prompt_context = prompt_context.replace("'", "")
  
    prompt = f"""
      'Eres un asistente experto en extraer información de currículums.
       Responde a la pregunta basándote en el contexto proporcionado. Sé conciso y no inventes información.
       Si no tienes la información, simplemente dilo.
       
      Contexto: {prompt_context}
      
      Pregunta:  
       {pregunta} 
       
       Respuesta: '
       """
    
    # Obtener URLs
    file_data = []
    for path in unique_paths:
        # Filtramos fragmentos relacionados con este archivo
        file_chunks = df_context[df_context['RELATIVE_PATH'] == path]
        
        # Obtenemos URL
        cmd2 = f"SELECT GET_PRESIGNED_URL(@curriculum_schema.cvs_to_ocr, '{path}', 3600) AS URL_LINK FROM directory(@curriculum_schema.cvs_to_ocr)"
        df_url_link = session.sql(cmd2).to_pandas()
        url_link = df_url_link._get_value(0, 'URL_LINK')
        
        # Crear estructura de datos con información relevante
        file_data.append({
            "path": path, 
            "url": url_link,
            "chunks": file_chunks['CHUNK'].tolist(),
            "similarity": file_chunks['SIMILARITY'].max()
        })
    
    # Ordenamos por relevancia (mayor similitud)
    file_data = sorted(file_data, key=lambda x: x['similarity'], reverse=True)
    
    return prompt, file_data

def analyze_cv(file_info):
    """Analiza el contenido de un CV basado en los fragmentos de texto"""
    # Concatenar todos los fragmentos de este archivo
    full_text = " ".join(file_info["chunks"])
    
    # Prompt para análisis del cv
    analysis_prompt = f"""
    'Eres un experto en análisis de currículums. 
     Analiza el siguiente fragmento de texto extraído de un CV y proporciona un resumen estructurado 
     con los siguientes elementos:
     
     1. PERFIL: Breve descripción del perfil profesional
     2. EXPERIENCIA: Puntos clave de su experiencia laboral
     3. HABILIDADES: Lista de habilidades principales identificadas
     4. FORMACIÓN: Información sobre su formación académica
     
     Texto del CV: {full_text} 
     
     Análisis: '
    """
    cmd = "SELECT SNOWFLAKE.CORTEX.COMPLETE(?, ?) AS analysis"
    df_analysis = session.sql(cmd, params=[model, analysis_prompt]).collect()
    return df_analysis[0]['ANALYSIS']

def complete(pregunta):
    prompt, file_data = create_prompt(pregunta)
    #Utilizamos la función de cortex "complete" para que complete también el análisis del cv
    cmd = """
             SELECT SNOWFLAKE.CORTEX.COMPLETE(?, ?) AS response 
           """
    df_response = session.sql(cmd, params=[model, prompt]).collect()
    
    # Analizar cada CV único
    for file in file_data:
        file["analysis"] = analyze_cv(file)
        # Calcular la confianza basada en la similitud
        file["confidence"] = min(100, int(file["similarity"] * 100))
    
    return df_response[0]['RESPONSE'], file_data


def display_response(pregunta):
    with st.status("Analizando documentos...") as status:
        try:
            response, file_data = complete(pregunta)
            
            # Mostrar la respuesta principal
            st.header("Respuesta")
            st.info(response)
            
            # Actualizar el estado
            status.update(label="¡Análisis completado!", state="complete", expanded=False)
            
            # Mostrar documentos consultados
            st.header(f"Documentos consultados ({len(file_data)})")
            
            # Mostrar cada documento en secciones separadas
            for i, file in enumerate(file_data):
                st.subheader(f"Documento {i+1}: {file['path'].split('/')[-1]}")
                st.markdown("### 📄 Análisis del CV")
                st.markdown(file["analysis"])
                
                # Sección de relevancia
                st.markdown("### 🎯 Relevancia para la consulta")
                col1, col2 = st.columns([3, 1])
                
                with col1:
                    st.progress(file["confidence"]/100)
                
                with col2:
                    # Mostrar porcentaje de relevancia
                    st.markdown(f"<h2 style='text-align: center; color: {'green' if file['confidence'] > 60 else 'orange' if file['confidence'] > 20 else 'red'};'>{file['confidence']}%</h2>", unsafe_allow_html=True)
                
                # Descripción sobre la relevancia
                relevance_description = ""
                if file["confidence"] > 60:
                    relevance_description = "Este documento es altamente relevante para tu consulta."
                elif file["confidence"] > 40:
                    relevance_description = "Este documento contiene información bastante relevante para tu consulta."
                elif file["confidence"] > 20:
                    relevance_description = "Este documento es moderadamente relevante para tu consulta."
                else:
                    relevance_description = "Este documento tiene baja relevancia, pero puede contener información contextual útil."
                
                st.markdown(f"_{relevance_description}_")
                st.markdown("---")
                
        except Exception as e:
            st.error(f"Error al procesar la consulta: {str(e)}")
            status.update(label="Error en el procesamiento", state="error", expanded=True)

# Estilos
st.markdown("""
<style>
    h1, h2, h3, h4 {
        color: #1E88E5;
    }
    .relevance-high {
        background-color: #D5F5E3;
        padding: 10px;
        border-radius: 5px;
        border-left: 5px solid #2ECC71;
    }
    .relevance-medium {
        background-color: #FCF3CF;
        padding: 10px;
        border-radius: 5px;
        border-left: 5px solid #F1C40F;
    }
    .relevance-low {
        background-color: #FADBD8;
        padding: 10px;
        border-radius: 5px;
        border-left: 5px solid #E74C3C;
    }
    .stProgress > div > div {
        height: 20px;
    }
</style>
""", unsafe_allow_html=True)

# Código principal
st.title("📑 Consulta Inteligente de Currículums")
st.write("Haz preguntas sobre los currículums y obtén respuestas basadas en la información extraída mediante OCR")

# Interfaz de búsqueda
col1, col2 = st.columns([5, 1])
with col1:
    pregunta = st.text_input("Ingresa tu pregunta", placeholder="¿Qué experiencia tiene este candidato?")
with col2:
    buscar = st.button("🔍 Buscar", use_container_width=True)

if pregunta and buscar:
    display_response(pregunta)

# Escalabilidad y Líneas Futuras de Investigación

## Áreas de Exploración Post-POC

Esta prueba de concepto demuestra el potencial del análisis de CVs mediante OCR y arquitectura RAG. Cosas a mejorar:

### Optimización de Recursos

- **Persistencia de análisis**: Almacenar resultados de análisis previos para evitar reprocesamiento de documentos sin cambios.
- **Procesamiento asincrónico**: Implementar mecanismos de procesamiento en segundo plano para grandes volúmenes de CVs.

### Mejoras Técnicas

- **Chunking semántico**: Segmentar documentos según secciones relevantes de CVs (experiencia, educación, habilidades).
- **Indexación vectorial optimizada**: Explorar técnicas avanzadas de indexación para búsquedas más eficientes.

### Seguridad Avanzada

- **Enmascaramiento contextual**: Políticas dinámicas basadas en el tipo de información detectada.
- **Control de acceso granular**: Seguridad a nivel de fila y columna para diversos perfiles de usuarios.
- **Clasificación automática**: Detección y etiquetado de información sensible en CVs.

### Ampliación de Capacidades

- **Procesamiento multilingüe**: Extender el OCR para soportar diversos idiomas y formatos internacionales.
- **Detección de tipos de documento**: Clasificación automática de formatos y estructuras de CV.
- **Experiencia interactiva**: Mejorar la interfaz para permitir exploración y refinamiento de resultados.
- **Métricas de calidad**: Implementar sistemas de monitoreo para evaluar la precisión del OCR y RAG.
- **Cumplimiento normativo**: Adaptación a diferentes marcos regulatorios (GDPR, CCPA, etc.).

Esta POC sienta las bases para explorar estas interesantes direcciones.